# I. US County Race Population

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity', 'FullRate_Black', 'FullRate_White', 'FullRate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report


In [2]:
path = 'DataMerge/Data/C-AnnualCountyPopulation/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [3]:
STATE_VACCINE = {}

In [4]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)


Enter the date of collecting data (Today's data in YYYY-MM-DD):
2021-05-20
Hello, 2021-05-20


# II. API States

## A. Illinois* (Done) XX


https://www.dph.illinois.gov/covid19/vaccinedata?county=Illinois


* Only Number


* Admin-Count (People at least One)

* Full-Count (Fully Covered)



White_Num = White_noHis_Num


In [5]:
from datetime import datetime 
import numpy as np

import requests
import time

In [6]:
state = 'Illinois'


def get_Illinois_Raw_and_postprocess(US_COUNTY_POPULATION):
    

    il_county_list = [i for i in US_COUNTY_POPULATION[US_COUNTY_POPULATION['State'] == 'Illinois']['County'].values]

    il_county_list.sort()

    L = []
    error_list = []
    for c in il_county_list:
        # c = 'Mchenry'
        api_url = 'https://idph.illinois.gov/DPHPublicInformation/api/covidvaccine/getVaccineAdministrationDemos?countyname={}'.format(c)

        resp = requests.get(api_url)# .to_json()
        data = resp.json() # resp
        if sum([len(v) for k, v in data.items()]) == 0:
            error_list.append(c)
            print(c)
            continue

        time.sleep(0.1)
        try:
            D = {}
            D['County'] = c
            for d in data['Race']:
                D[d['Race'] + '-AdminCount'] = d['AdministeredCount']
                D[d['Race'] + '-FullyVaccn'] = d['PersonsFullyVaccinated']
                date = d['Report_Date']

            D['Date'] = date.split('T')[0]
            L.append(D)

        except:
            print('No Data')
            print(c)

    IL_County_Vaccine_Race = pd.DataFrame(L)
    return IL_County_Vaccine_Race


IL_County_Vaccine_Race = get_Illinois_Raw_and_postprocess(US_COUNTY_POPULATION)
# IL_County_Vaccine_Race.to_csv('FinalResult/Raw_Illinois.csv')
print(IL_County_Vaccine_Race.shape)
IL_County_Vaccine_Race.head()

(102, 18)


,County,Hispanic or Latino-AdminCount,Hispanic or Latino-FullyVaccn,Asian-AdminCount,Asian-FullyVaccn,Other race-AdminCount,Other race-FullyVaccn,Black or African-American-AdminCount,Black or African-American-FullyVaccn,White-AdminCount,White-FullyVaccn,American Indian or Alaska Nati-AdminCount,American Indian or Alaska Nati-FullyVaccn,Native Hawaiian or Other Pacif-AdminCount,Native Hawaiian or Other Pacif-FullyVaccn,Unknown-AdminCount,Unknown-FullyVaccn,Date
0,Adams,533,288,376.0,202.0,153,88,1169,644,41784,22523,55.0,28.0,57.0,29.0,1040,548,2021-05-19
1,Alexander,9,4,NaN,NaN,8,4,511,236,894,423,5.0,1.0,2.0,1.0,102,50,2021-05-19
2,Bond,162,66,44.0,18.0,132,88,168,72,9149,4433,3.0,1.0,10.0,3.0,281,129,2021-05-19
3,Boone,5531,2189,470.0,218.0,742,362,600,273,29523,13842,131.0,63.0,49.0,23.0,1982,936,2021-05-19
4,Brown,11,6,6.0,3.0,1,1,26,15,2774,1379,6.0,3.0,NaN,NaN,68,33,2021-05-19


In [7]:
IL_County_Vaccine_Race.iloc[0].to_dict()

{'County': 'Adams',
 'Hispanic or Latino-AdminCount': 533,
 'Hispanic or Latino-FullyVaccn': 288,
 'Asian-AdminCount': 376.0,
 'Asian-FullyVaccn': 202.0,
 'Other race-AdminCount': 153,
 'Other race-FullyVaccn': 88,
 'Black or African-American-AdminCount': 1169,
 'Black or African-American-FullyVaccn': 644,
 'White-AdminCount': 41784,
 'White-FullyVaccn': 22523,
 'American Indian or Alaska Nati-AdminCount': 55.0,
 'American Indian or Alaska Nati-FullyVaccn': 28.0,
 'Native Hawaiian or Other Pacif-AdminCount': 57.0,
 'Native Hawaiian or Other Pacif-FullyVaccn': 29.0,
 'Unknown-AdminCount': 1040,
 'Unknown-FullyVaccn': 548,
 'Date': '2021-05-19'}

In [8]:


def standardize_Illinois(IL_County_Vaccine_Race, US_COUNTY_POPULATION):
    
    state = 'Illinois'

    new_L = []


    for idx, row in IL_County_Vaccine_Race.iterrows(): # change this
        d = row.to_dict()

        new_d = {}


        # GEO Information
        new_d['State'] = state
        new_d['County'] = d['County']


        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        # At Least One
        new_d['Numb_White'] = d['White-AdminCount']
        # new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Black or African-American-AdminCount']
        # new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Asian-AdminCount']
        # new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = d['American Indian or Alaska Nati-AdminCount']
        # new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Other race-AdminCount']
        
       
        
        
        
        # Fully Vaccination
        new_d['FullNumb_Hispa'] = d['Hispanic or Latino-FullyVaccn']
        
        new_d['FullNumb_White'] = d['White-FullyVaccn']
        # new_d['Rate_White'] = np.nan

        new_d['FullNumb_Black'] = d['Black or African-American-FullyVaccn']
        # new_d['Rate_Black'] = np.nan

        new_d['FullNumb_Asian'] = d['Asian-FullyVaccn']
        # new_d['Rate_Asian'] = np.nan

        new_d['FullNumb_AIndA'] = d['American Indian or Alaska Nati-FullyVaccn']
        # new_d['Rate_AIndA'] = np.nan

        new_d['FullNumb_Other'] = d['Other race-FullyVaccn']
        
        new_d['FullNumb_Hispa'] = d['Hispanic or Latino-FullyVaccn']
        
        
        

        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)

    
    Data = pd.DataFrame(new_L)
    
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian', 'AIndA']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate

        Rate = T['FullNumb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['FullRate_' + Race] = Rate

        
        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    return T
        
        


T = standardize_Illinois(IL_County_Vaccine_Race, US_COUNTY_POPULATION)
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

(102, 32)


,State,County,Geographic Area Name,Date,FullNumb_AIndA,FullNumb_Asian,FullNumb_Black,FullNumb_Hispa,FullNumb_Other,FullNumb_White,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Illinois,Adams,"Adams, Illinois",2021-05-20,28.0,202.0,644,288,88,22523,...,0.688392,190,395,615,830,2557,3554,60698,61991,65435
1,Illinois,Alexander,"Alexander, Illinois",2021-05-20,1.0,NaN,236,4,4,423,...,0.239037,26,62,17,36,1830,1917,3740,3874,5761
2,Illinois,Bond,"Bond, Illinois",2021-05-20,1.0,18.0,72,66,88,4433,...,0.615100,110,183,134,187,1041,1195,14874,15113,16426
3,Illinois,Boone,"Boone, Illinois",2021-05-20,63.0,218.0,273,2189,362,13842,...,0.596063,420,1006,785,1052,1503,1984,49530,50712,53544
4,Illinois,Brown,"Brown, Illinois",2021-05-20,3.0,3.0,15,6,1,1379,...,0.537076,87,107,21,43,1242,1276,5165,5212,6578


In [9]:
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']

T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']

STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,102.0,0.130924,0.070560,0.011294,0.077300,0.131942,0.178640,0.472973
1,FullRate_White,102.0,0.298056,0.055363,0.113102,0.262241,0.301118,0.341900,0.445308
2,Rate_Black,102.0,0.282069,0.151872,0.018480,0.174668,0.280767,0.371927,1.013514
3,Rate_White,102.0,0.609431,0.117727,0.239037,0.521963,0.619903,0.692829,0.920960
4,FullRate_Parity,102.0,0.167132,0.074430,-0.166198,0.129831,0.169090,0.216881,0.322707
5,Rate_Parity,102.0,0.327363,0.154183,-0.380264,0.256996,0.336082,0.433222,0.626430


# III. Table States

## B. Texas* (Done) XX

download from: 
https://www.dshs.texas.gov/immunize/covid19/COVID-19-Vaccine-Data-by-County.xls


which is from: https://www.dshs.texas.gov/coronavirus/immunize/vaccine.aspx


Only Number, No Rate.


Three conceptions:

* Administered, 

* Vaccinated

* Fully Vaccinated

In [10]:
def get_Texas_Raw():
    TEXAS_URL = 'https://www.dshs.texas.gov/immunize/covid19/COVID-19-Vaccine-Data-by-County.xls'
    RawData = pd.read_excel(TEXAS_URL, sheet_name='By County, Race')
    return RawData


RawData = get_Texas_Raw()
RawData.columns = [i.strip() for i in RawData.columns]
RawData

,County Name,Race/Ethnicity,People Vaccinated with at least One Dose,People Fully Vaccinated,Doses Administered
0,Anderson,Asian,166,118,272
1,Anderson,Black,3148,1041,3996
2,Anderson,Hispanic,1273,838,1988
3,Anderson,Other,1844,529,2321
4,Anderson,Unknown,1118,841,1913
...,...,...,...,...,...
1507,Other,Hispanic,41538,23896,62068
1508,Other,Other,12352,7935,19040
1509,Other,Unknown,14137,9046,21958
1510,Other,White,65941,42091,101985


In [11]:
def postprocess_Texas_Raw(RawData):
    RawDataGroups = RawData.groupby('County Name')
    L = []
    for county, record in RawDataGroups:
        D = {}

        D['County'] = county

        for idx, row in record.iterrows():
            d = row.to_dict()
            # print(d)
            race = d['Race/Ethnicity'] 

            D[race+'_DoesesAdmin'] = d['Doses Administered']
            D[race+'_Vaccinated']  = d['People Vaccinated with at least One Dose']
            D[race+'_FullVacc']     = d['People Fully Vaccinated']
        L.append(D)


    Texas_County_Vaccine_Race = pd.DataFrame(L)
    
    return Texas_County_Vaccine_Race


Texas_County_Vaccine_Race = postprocess_Texas_Raw(RawData)
# Texas_County_Vaccine_Race.to_csv('FinalResult/Post_Texas.csv')
print(Texas_County_Vaccine_Race.shape)
Texas_County_Vaccine_Race.head() # 254 counties + 1 total

(256, 22)


,County,Asian_DoesesAdmin,Asian_Vaccinated,Asian_FullVacc,Black_DoesesAdmin,Black_Vaccinated,Black_FullVacc,Hispanic_DoesesAdmin,Hispanic_Vaccinated,Hispanic_FullVacc,...,Other_FullVacc,Unknown_DoesesAdmin,Unknown_Vaccinated,Unknown_FullVacc,White_DoesesAdmin,White_Vaccinated,White_FullVacc,Total_DoesesAdmin,Total_Vaccinated,Total_FullVacc
0,Anderson,272.0,166.0,118.0,3996.0,3148.0,1041.0,1988.0,1273.0,838.0,...,529.0,1913.0,1118.0,841.0,12994.0,8405.0,5685.0,NaN,NaN,NaN
1,Andrews,52.0,31.0,21.0,79.0,43.0,37.0,4443.0,2415.0,2039.0,...,103.0,2252.0,1274.0,1120.0,3880.0,2094.0,1824.0,NaN,NaN,NaN
2,Angelina,542.0,307.0,246.0,5491.0,3131.0,2580.0,7981.0,4532.0,3698.0,...,1140.0,7422.0,4008.0,3519.0,22980.0,12906.0,11103.0,NaN,NaN,NaN
3,Aransas,353.0,205.0,156.0,108.0,65.0,46.0,2885.0,1670.0,1351.0,...,1027.0,2328.0,1257.0,1086.0,11843.0,6749.0,5542.0,NaN,NaN,NaN
4,Archer,17.0,9.0,9.0,27.0,19.0,14.0,225.0,137.0,112.0,...,463.0,1145.0,602.0,554.0,3231.0,1807.0,1665.0,NaN,NaN,NaN


In [42]:
Texas_County_Vaccine_Race.columns

Index(['County', 'Asian_DoesesAdmin', 'Asian_Vaccinated', 'Asian_FullVacc',
       'Black_DoesesAdmin', 'Black_Vaccinated', 'Black_FullVacc',
       'Hispanic_DoesesAdmin', 'Hispanic_Vaccinated', 'Hispanic_FullVacc',
       'Other_DoesesAdmin', 'Other_Vaccinated', 'Other_FullVacc',
       'Unknown_DoesesAdmin', 'Unknown_Vaccinated', 'Unknown_FullVacc',
       'White_DoesesAdmin', 'White_Vaccinated', 'White_FullVacc',
       'Total_DoesesAdmin', 'Total_Vaccinated', 'Total_FullVacc'],
      dtype='object')

In [13]:
import numpy as np
def standardize_Texas(Texas_County_Vaccine_Race, US_COUNTY_POPULATION):
    state = 'Texas'

    new_L = []


    for idx, row in Texas_County_Vaccine_Race.iterrows():
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = d['County']

        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

        new_d['Numb_White'] = d['White_Vaccinated']
        
        new_d['Numb_Black'] = d['Black_Vaccinated']
        
        new_d['Numb_Asian'] = d['Asian_Vaccinated']
        
        new_d['Numb_Other'] = d['Other_Vaccinated']
        
        new_d['Numb_Hispa'] = d['Hispanic_Vaccinated']
        
        
        new_d['FullNumb_White'] = d['White_FullVacc']
        
        new_d['FullNumb_Black'] = d['Black_FullVacc']
        
        new_d['FullNumb_Asian'] = d['Asian_FullVacc']
        
        new_d['FullNumb_Other'] = d['Other_FullVacc']
        
        new_d['FullNumb_Hispa'] = d['Hispanic_FullVacc']
        

        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)
    # StateCountyVaccine[state] = Data
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)



    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate
        
        Rate = T['FullNumb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['FullRate_' + Race] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return T



T = standardize_Texas(Texas_County_Vaccine_Race, US_COUNTY_POPULATION)

state = 'Texas'
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Other, Texas', 'Grand Total, Texas']
(254, 29)


,State,County,Geographic Area Name,Date,FullNumb_Asian,FullNumb_Black,FullNumb_Hispa,FullNumb_Other,FullNumb_White,FullRate_Asian,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Texas,Anderson,"Anderson, Texas",2021-05-20,118.0,1041.0,838.0,529.0,5685.0,0.219331,...,0.195306,428,747,538,757,12628,13200,43035,43927,57735
1,Texas,Andrews,"Andrews, Texas",2021-05-20,21.0,37.0,2039.0,103.0,1824.0,0.161538,...,0.118701,274,392,130,171,382,502,17641,17900,18705
2,Texas,Angelina,"Angelina, Texas",2021-05-20,246.0,2580.0,3698.0,1140.0,11103.0,0.266522,...,0.183778,665,1040,923,1166,13595,14256,70226,71385,86715
3,Texas,Aransas,"Aransas, Texas",2021-05-20,156.0,46.0,1351.0,1027.0,5542.0,0.317719,...,0.309006,308,542,491,564,406,554,21841,22252,23510
4,Texas,Archer,"Archer, Texas",2021-05-20,9.0,14.0,112.0,463.0,1665.0,0.219512,...,0.222045,131,200,41,66,96,153,8138,8264,8553


In [15]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,247.0,0.118574,0.065933,0.000000,0.065640,0.113300,0.166311,0.281543
1,FullRate_White,254.0,0.148113,0.058892,0.018203,0.111210,0.150763,0.184150,0.320708
2,Rate_Black,247.0,0.165013,0.084993,0.014815,0.102421,0.164226,0.218315,0.697479
3,Rate_White,254.0,0.175577,0.066897,0.023512,0.131270,0.183936,0.216119,0.343335
4,Rate_Parity,247.0,0.011505,0.087982,-0.660907,-0.037146,0.003101,0.052567,0.256978
5,FullRate_Parity,247.0,0.030255,0.063156,-0.112150,-0.013295,0.020850,0.065323,0.210501


## B. Pennsylvania* (Done)

67 counties

Ethnicity
https://data.pa.gov/Health/COVID-19-Vaccinations-by-Ethnicity-Current-County-/7ruj-m7k6 

Race
https://data.pa.gov/Health/COVID-19-Vaccinations-by-Race-Current-County-Healt/x5z9-57ub


Only Number

In [16]:

Penn_Ethn_URL = 'https://data.pa.gov/api/views/7ruj-m7k6/rows.csv?accessType=DOWNLOAD'
Penn_Race_URL = 'https://data.pa.gov/api/views/x5z9-57ub/rows.csv?accessType=DOWNLOAD'

Ethn = pd.read_csv(Penn_Ethn_URL).rename(columns = {'Ethnicity':'Item', 'County Name': 'County'})
Race = pd.read_csv(Penn_Race_URL).rename(columns = {'Race':'Item'})


RawData = pd.concat([Race, Ethn])

RawData.columns


Index(['County Name', 'Partially Covered African American',
       'Partially Covered Asian', 'Partially Covered Native American',
       'Partially Covered Pacific Islander',
       'Partially Covered Multiple Other', 'Partially Covered White',
       'Partially Covered  Unknown', 'Fully Covered African American',
       'Fully Covered Asian', 'Fully Covered Native American',
       'Fully Covered Pacific Islander', 'Fully Covered Multiple Other',
       'Fully Covered White', 'Fully Covered Unknown', 'County',
       'Partially Covered Hispanic', 'Partially Covered Not Hispanic',
       'Partially Covered Unknown', 'Fully Covered Hispanic',
       'Fully Covered Not Hispanic'],
      dtype='object')

In [17]:
Race.head()

,County Name,Partially Covered African American,Partially Covered Asian,Partially Covered Native American,Partially Covered Pacific Islander,Partially Covered Multiple Other,Partially Covered White,Partially Covered Unknown,Fully Covered African American,Fully Covered Asian,Fully Covered Native American,Fully Covered Pacific Islander,Fully Covered Multiple Other,Fully Covered White,Fully Covered Unknown
0,BLAIR,131.0,15.0,NaN,18.0,567,6590,1272,215.0,18.0,NaN,24.0,3741,25534,6431
1,DAUPHIN,4678.0,78.0,5.0,101.0,3865,23819,6559,7856.0,193.0,16.0,170.0,9276,58212,8179
2,CENTRE,416.0,49.0,8.0,68.0,686,11499,3865,1625.0,198.0,15.0,204.0,2724,44190,11792
3,MONTGOMERY,13210.0,1062.0,47.0,530.0,10104,166008,29757,12819.0,2694.0,147.0,672.0,19342,167138,38225
4,LYCOMING,285.0,12.0,NaN,26.0,944,7679,1323,776.0,14.0,NaN,25.0,5613,25994,3056


In [18]:
import pandas as pd

Pennsylvania_County_Vaccine_Race =  Race                         
Pennsylvania_County_Vaccine_Race

,County Name,Partially Covered African American,Partially Covered Asian,Partially Covered Native American,Partially Covered Pacific Islander,Partially Covered Multiple Other,Partially Covered White,Partially Covered Unknown,Fully Covered African American,Fully Covered Asian,Fully Covered Native American,Fully Covered Pacific Islander,Fully Covered Multiple Other,Fully Covered White,Fully Covered Unknown
0,BLAIR,131.0,15.0,NaN,18.0,567,6590,1272,215.0,18.0,NaN,24.0,3741,25534,6431
1,DAUPHIN,4678.0,78.0,5.0,101.0,3865,23819,6559,7856.0,193.0,16.0,170.0,9276,58212,8179
2,CENTRE,416.0,49.0,8.0,68.0,686,11499,3865,1625.0,198.0,15.0,204.0,2724,44190,11792
3,MONTGOMERY,13210.0,1062.0,47.0,530.0,10104,166008,29757,12819.0,2694.0,147.0,672.0,19342,167138,38225
4,LYCOMING,285.0,12.0,NaN,26.0,944,7679,1323,776.0,14.0,NaN,25.0,5613,25994,3056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,CLINTON,24.0,NaN,NaN,NaN,241,1934,326,35.0,NaN,NaN,NaN,1185,8563,662
64,NORTHUMBERLAND,79.0,NaN,NaN,11.0,359,5567,1010,707.0,6.0,NaN,16.0,1582,26323,2478
65,SUSQUEHANNA,15.0,NaN,NaN,NaN,127,2320,236,28.0,6.0,NaN,5.0,502,8768,827
66,LAWRENCE,155.0,NaN,NaN,10.0,440,4341,1001,422.0,10.0,NaN,16.0,2395,19438,3800


In [19]:
def standardize_Penn(Pennsylvania_County_Vaccine_Race, US_COUNTY_POPULATION):

    state = 'Pennsylvania' ## 

    new_L = []

    for idx, row in Pennsylvania_County_Vaccine_Race.iterrows(): ## 
        d = row.to_dict()

        new_d = {}
        new_d['State'] = state
        new_d['County'] = d['County Name'].lower().capitalize()

        RenameCounties = {
            'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        new_d['Numb_White'] = d['Partially Covered White']                  + d['Fully Covered White']   
        
        new_d['Numb_Black'] = d['Partially Covered African American']       + d['Fully Covered African American']
        
        new_d['Numb_Asian'] = d['Partially Covered Asian']                  + d['Fully Covered Asian']                  
         
        new_d['FullNumb_White'] = d['Fully Covered White']   
        
        new_d['FullNumb_Black'] = d['Fully Covered African American']
        
        new_d['FullNumb_Asian'] = d['Fully Covered Asian']                  
         


        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)
    
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)

    
    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')
    print(T.columns)
    for Race in ['White', 'Black', 'Asian']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate
        
        Rate = T['FullNumb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['FullRate_' + Race] = Rate



        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return Data, T

In [20]:
Data, T = standardize_Penn(Pennsylvania_County_Vaccine_Race, US_COUNTY_POPULATION)
state = 'Pennsylvania'
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Out-of-state, Pennsylvania']
Index(['State', 'County', 'Geographic Area Name', 'Numb_White', 'Numb_Black',
       'Numb_Asian', 'FullNumb_White', 'FullNumb_Black', 'FullNumb_Asian',
       'Date', 'Total_Whole', 'Total_White', 'Total_Black', 'Total_AIndA',
       'Total_Asian', 'Total_WhiteC', 'Total_BlackC', 'Total_AIndAC',
       'Total_AsianC'],
      dtype='object')
(67, 25)


,State,County,Geographic Area Name,Date,FullNumb_Asian,FullNumb_Black,FullNumb_White,FullRate_Asian,FullRate_Black,FullRate_White,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Pennsylvania,Blair,"Blair, Pennsylvania",2021-05-20,18.0,215.0,25534,0.021077,0.087327,0.219304,...,0.275904,197,571,854,1224,2462,3648,116432,118182,121829
1,Pennsylvania,Dauphin,"Dauphin, Pennsylvania",2021-05-20,193.0,7856.0,58212,0.013020,0.146943,0.292680,...,0.412438,1248,3027,14823,16481,53463,60954,198893,207697,278299
2,Pennsylvania,Centre,"Centre, Pennsylvania",2021-05-20,198.0,1625.0,44190,0.019017,0.263414,0.309692,...,0.390280,252,847,10412,11686,6169,7253,142690,145215,162385
3,Pennsylvania,Montgomery,"Montgomery, Pennsylvania",2021-05-20,2694.0,12819.0,167138,0.040038,0.154438,0.253489,...,0.505264,1977,5696,67286,73971,83004,94306,659350,676454,830915
4,Pennsylvania,Lycoming,"Lycoming, Pennsylvania",2021-05-20,14.0,776.0,25994,0.018373,0.142307,0.249451,...,0.323142,326,800,762,1099,5453,7301,104205,106608,113299


In [22]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,64.0,0.155059,0.094396,0.014620,0.099091,0.130514,0.181509,0.522936
1,FullRate_White,67.0,0.274974,0.067876,0.045139,0.230359,0.271743,0.310769,0.473257
2,Rate_Black,61.0,0.208662,0.089413,0.035088,0.152482,0.194572,0.263782,0.448995
3,Rate_White,67.0,0.360150,0.090239,0.066519,0.298920,0.359684,0.420880,0.547521
4,Rate_Parity,61.0,0.149546,0.089310,-0.144379,0.109791,0.152998,0.196955,0.311353
5,FullRate_Parity,64.0,0.120361,0.087218,-0.196035,0.102430,0.132779,0.169651,0.261904


## B. Indiana* (Done)

https://hub.mph.in.gov/dataset/covid-19-vaccinations-demographics-by-county-and-district/resource/82d99020-093f-41ac-95c7-d3c335b8c2ba

In [23]:
# path = 'TMP/Indiana_Mar26_county-vaccination-demographics.xlsx'
# path = 'TMP/Indiana/Indiana_April6_county-vaccination-demographics.xlsx'

url = 'https://hub.mph.in.gov/dataset/145a43b2-28e5-4bf1-ad86-123d07fddb55/resource/82d99020-093f-41ac-95c7-d3c335b8c2ba/download/county-vaccination-demographics.xlsx'

RaceData = pd.read_excel(url, sheet_name = 'Race').rename(columns = {"race":"item"})
EthnicityData = pd.read_excel(url, sheet_name = 'Ethnicity').rename(columns = {"ethnicity":"item"})
Data = pd.concat([RaceData, EthnicityData])
Data.head()

,county,region,fips,item,first_dose_administered,second_dose_administered,single_dose_administered,all_doses_administered,fully_vaccinated,new_first_dose_administered,new_second_dose_administered,new_single_dose_administered,new_all_doses_administered,new_fully_vaccinated,current_as_of
0,Adams,3,18001,Asian,48,43,Suppressed,91,43,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,5/20/2021 5:00
1,Adams,3,18001,Black or African American,21,17,Suppressed,42,21,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,5/20/2021 5:00
2,Adams,3,18001,Other Race,191,167,37,395,204,Suppressed,Suppressed,Suppressed,5,Suppressed,5/20/2021 5:00
3,Adams,3,18001,Unknown,173,138,162,473,300,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,5/20/2021 5:00
4,Adams,3,18001,White,8315,7715,397,16427,8112,18,20,5,43,25,5/20/2021 5:00


In [26]:

# Data.columns


L= []

for county, record in Data.groupby('county'):
    # print(county)
    D = {}
    D['county'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        # print(d)
        race = d['item']
        try:
            D[race + '_' + 'AtLeastOneDose'] = float(d['all_doses_administered'])
        except:
            D[race + '_' + 'AtLeastOneDose'] = np.nan
            
        try:
            D[race + '_' + 'FullNum'] = float(d['fully_vaccinated'])
        except:
            D[race + '_' + 'FullNum'] = np.nan
    # print(D)
    L.append(D)

Indiana_County_Vaccine_Race = pd.DataFrame(L)



In [27]:
print(Indiana_County_Vaccine_Race.shape)
Indiana_County_Vaccine_Race.head() # 95 counties in Pennsylvania

(94, 15)


,county,Asian_AtLeastOneDose,Asian_FullNum,Black or African American_AtLeastOneDose,Black or African American_FullNum,Other Race_AtLeastOneDose,Other Race_FullNum,Unknown_AtLeastOneDose,Unknown_FullNum,White_AtLeastOneDose,White_FullNum,Hispanic or Latino_AtLeastOneDose,Hispanic or Latino_FullNum,Not Hispanic or Latino_AtLeastOneDose,Not Hispanic or Latino_FullNum
0,Adams,91.0,43.0,42.0,21.0,395.0,204.0,1118.0,630.0,16427.0,8112.0,485.0,238.0,15825.0,7812.0
1,Allen,10430.0,4686.0,17978.0,8409.0,13264.0,6096.0,26774.0,12774.0,210560.0,101525.0,9941.0,4459.0,226751.0,108560.0
2,Bartholomew,5394.0,2539.0,1003.0,469.0,2680.0,1191.0,2356.0,1110.0,56288.0,26895.0,2903.0,1253.0,62369.0,29738.0
3,Benton,12.0,6.0,24.0,12.0,146.0,85.0,198.0,98.0,4353.0,2007.0,217.0,122.0,4243.0,1959.0
4,Blackford,15.0,7.0,11.0,6.0,85.0,39.0,283.0,141.0,7258.0,3541.0,62.0,29.0,7189.0,3504.0


In [28]:
# Indiana_County_Vaccine_Race.to_csv('FinalResult/Indiana.csv')
print(Indiana_County_Vaccine_Race.shape)
Indiana_County_Vaccine_Race.head() # 95 counties in Pennsylvania

(94, 15)


,county,Asian_AtLeastOneDose,Asian_FullNum,Black or African American_AtLeastOneDose,Black or African American_FullNum,Other Race_AtLeastOneDose,Other Race_FullNum,Unknown_AtLeastOneDose,Unknown_FullNum,White_AtLeastOneDose,White_FullNum,Hispanic or Latino_AtLeastOneDose,Hispanic or Latino_FullNum,Not Hispanic or Latino_AtLeastOneDose,Not Hispanic or Latino_FullNum
0,Adams,91.0,43.0,42.0,21.0,395.0,204.0,1118.0,630.0,16427.0,8112.0,485.0,238.0,15825.0,7812.0
1,Allen,10430.0,4686.0,17978.0,8409.0,13264.0,6096.0,26774.0,12774.0,210560.0,101525.0,9941.0,4459.0,226751.0,108560.0
2,Bartholomew,5394.0,2539.0,1003.0,469.0,2680.0,1191.0,2356.0,1110.0,56288.0,26895.0,2903.0,1253.0,62369.0,29738.0
3,Benton,12.0,6.0,24.0,12.0,146.0,85.0,198.0,98.0,4353.0,2007.0,217.0,122.0,4243.0,1959.0
4,Blackford,15.0,7.0,11.0,6.0,85.0,39.0,283.0,141.0,7258.0,3541.0,62.0,29.0,7189.0,3504.0


In [29]:
# print(len(L))

import numpy as np

state = 'Indiana' ## 

new_L = []


for idx, row in Indiana_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['county']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['county'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Lagrange': 'LaGrange',
        'La Porte': 'LaPorte', 
        'De Kalb': 'DeKalb',
        
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = float(d['White_AtLeastOneDose'])
    
    new_d['Numb_Black'] = d['Black or African American_AtLeastOneDose']
    
    new_d['Numb_Asian'] = d['Asian_AtLeastOneDose']
    
    new_d['Numb_NaHis'] = d['Not Hispanic or Latino_AtLeastOneDose']
    
    new_d['Numb_Hispa'] = d['Hispanic or Latino_AtLeastOneDose']
    
    
    new_d['FullNumb_White'] = float(d['White_FullNum'])
    
    new_d['FullNumb_Black'] = d['Black or African American_FullNum']
    
    new_d['FullNumb_Asian'] = d['Asian_FullNum']
    
    new_d['FullNumb_NaHis'] = d['Not Hispanic or Latino_FullNum']
    
    new_d['FullNumb_Hispa'] = d['Hispanic or Latino_FullNum']
    
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)

not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['FullNumb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['FullRate_' + Race] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()


Not included Counties ['Unknown, Indiana', 'Out Of State, Indiana']
(92, 29)


,State,County,Geographic Area Name,Date,FullNumb_Asian,FullNumb_Black,FullNumb_Hispa,FullNumb_NaHis,FullNumb_White,FullRate_Asian,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Indiana,Adams,"Adams, Indiana",NaN,43.0,21.0,238.0,7812.0,8112.0,0.313869,...,0.469410,117,208,137,183,198,384,34995,35300,35777
1,Indiana,Allen,"Allen, Indiana",NaN,4686.0,8409.0,4459.0,108560.0,101525.0,0.268831,...,0.697219,1780,4346,17431,19785,45450,53961,302000,313605,379299
2,Indiana,Bartholomew,"Bartholomew, Indiana",NaN,2539.0,469.0,1253.0,29738.0,26895.0,0.355951,...,0.773081,390,806,7133,7427,1974,2756,72810,74147,83779
3,Indiana,Benton,"Benton, Indiana",NaN,6.0,12.0,122.0,1959.0,2007.0,0.285714,...,0.513386,28,69,21,31,97,165,8479,8595,8748
4,Indiana,Blackford,"Blackford, Indiana",NaN,7.0,6.0,29.0,3504.0,3541.0,0.104478,...,0.637898,31,111,67,98,90,172,11378,11560,11758


In [30]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
# STATE_VACCINE = {}
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

T

['Illinois', 'Texas', 'Pennsylvania', 'Indiana']


,State,County,Geographic Area Name,Date,FullNumb_Asian,FullNumb_Black,FullNumb_Hispa,FullNumb_NaHis,FullNumb_White,FullRate_Asian,...,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,Rate_Parity,FullRate_Parity
0,Indiana,Adams,"Adams, Indiana",NaN,43.0,21.0,238.0,7812.0,8112.0,0.313869,...,208,137,183,198,384,34995,35300,35777,0.257289,0.125744
1,Indiana,Allen,"Allen, Indiana",NaN,4686.0,8409.0,4459.0,108560.0,101525.0,0.268831,...,4346,17431,19785,45450,53961,302000,313605,379299,0.301663,0.151159
2,Indiana,Bartholomew,"Bartholomew, Indiana",NaN,2539.0,469.0,1253.0,29738.0,26895.0,0.355951,...,806,7133,7427,1974,2756,72810,74147,83779,0.264975,0.131797
3,Indiana,Benton,"Benton, Indiana",NaN,6.0,12.0,122.0,1959.0,2007.0,0.285714,...,69,21,31,97,165,8479,8595,8748,0.265963,0.112991
4,Indiana,Blackford,"Blackford, Indiana",NaN,7.0,6.0,29.0,3504.0,3541.0,0.104478,...,111,67,98,90,172,11378,11560,11758,0.515675,0.244548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Indiana,Washington,"Washington, Indiana",NaN,24.0,17.0,46.0,7063.0,7253.0,0.282353,...,208,85,157,143,246,27434,27718,28036,0.283822,0.145499
88,Indiana,Wayne,"Wayne, Indiana",NaN,271.0,789.0,478.0,19341.0,18303.0,0.380084,...,673,713,1041,3316,4618,59598,61394,65884,0.132073,0.069170
89,Indiana,Wells,"Wells, Indiana",NaN,40.0,23.0,114.0,7642.0,7882.0,0.211640,...,201,189,248,249,429,27415,27754,28296,0.388513,0.195137
90,Indiana,White,"White, Indiana",NaN,31.0,13.0,262.0,8262.0,8445.0,0.227941,...,327,136,202,217,333,23236,23549,24102,0.606150,0.303537


In [31]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,89.0,0.175045,0.102368,0.050000,0.110000,0.171994,0.210841,0.832898
1,FullRate_White,92.0,0.311703,0.062741,0.149345,0.267380,0.309197,0.345720,0.494419
2,Rate_Black,90.0,0.371284,0.230152,0.053691,0.224396,0.351936,0.443820,1.937337
3,Rate_White,92.0,0.635823,0.127857,0.307123,0.542751,0.630369,0.713156,0.976631
4,Rate_Parity,90.0,0.268588,0.201569,-1.280970,0.213132,0.279290,0.353258,0.606150
5,FullRate_Parity,89.0,0.139888,0.089823,-0.515960,0.108180,0.145039,0.179915,0.303537


## B. Virginia* (Done) XX

https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-Vaccines-DosesAdmini/u5ru-3khs

In [32]:
import requests
from  sodapy import Socrata


# Today_Date = str(datetime.now().date())

##########################################
report_date = Today_Date + 'T00:00:00.000'
##########################################

client = Socrata("data.virginia.gov", None)

url = 'https://data.virginia.gov/resource/u5ru-3khs.json?report_date=' + report_date + 'T00:00:00.000?limit=3000'

results = client.get("u5ru-3khs", limit=5000, report_date = report_date)

# Convert to pandas DataFrame
RawData = pd.DataFrame.from_records(results)

In [33]:
while len(RawData) == 0:
    print('Error in Date, Please Change Date:')
    ##########################################
    new_date_for_virgina = input()
    report_date = new_date_for_virgina + 'T00:00:00.000'
    ##########################################

    client = Socrata("data.virginia.gov", None)

    url = 'https://data.virginia.gov/resource/u5ru-3khs.json?report_date=' + report_date + 'T00:00:00.000?limit=3000'

    results = client.get("u5ru-3khs", limit=5000, report_date = report_date)

    # Convert to pandas DataFrame
    RawData = pd.DataFrame.from_records(results)
    
RawData.head()

,report_date,fips,locality,health_district,health_region,vaccination_status,race_and_ethnicity,people_by_vaccination_status_count
0,2021-05-20T00:00:00.000,51610,Falls Church,Fairfax,Northern,Not Reported,Latino,0
1,2021-05-20T00:00:00.000,51071,Giles,New River,Southwest,Not Reported,White,0
2,2021-05-20T00:00:00.000,51075,Goochland,Chickahominy,Central,At Least One Dose,Latino,304
3,2021-05-20T00:00:00.000,51027,Buchanan,Cumberland Plateau,Southwest,Fully Vaccinated,Latino,41
4,2021-05-20T00:00:00.000,51111,Lunenburg,Piedmont,Central,Fully Vaccinated,Other Race,27


In [34]:
RawDataGroups = RawData.groupby('locality')


L = []
for county, record in RawDataGroups:
    D = {}

    D['County'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        race = d['race_and_ethnicity'] 
        
        d = row.to_dict()
        # print(d)
        D[race + '_' + d['vaccination_status']] = float(d['people_by_vaccination_status_count'])
        

    L.append(D)

    
Virginia_County_Vaccine_Race = pd.DataFrame(L)

In [35]:
# Virginia_County_Vaccine_Race.to_csv('FinalResult/Vriginia.csv')
print(Virginia_County_Vaccine_Race.shape)
Virginia_County_Vaccine_Race.head() # 67 counties in Pennsylvania

(135, 25)


,County,Black_Not Reported,Latino_Not Reported,Other Race_At Least One Dose,Two or more races_Fully Vaccinated,Not Reported_Fully Vaccinated,Latino_Fully Vaccinated,Black_Fully Vaccinated,Other Race_Not Reported,Other Race_Fully Vaccinated,...,Native American_Not Reported,Two or more races_At Least One Dose,White_At Least One Dose,Latino_At Least One Dose,Black_At Least One Dose,Two or more races_Not Reported,Asian or Pacific Islander_Not Reported,White_Not Reported,Not Reported_Not Reported,Not Reported_At Least One Dose
0,Accomack,0.0,0.0,35.0,0.0,2654.0,492.0,2018.0,0.0,26.0,...,0.0,0.0,7047.0,740.0,2479.0,0.0,0.0,0.0,0.0,3479.0
1,Albemarle,0.0,0.0,2838.0,0.0,14687.0,2471.0,2896.0,0.0,2388.0,...,0.0,0.0,37174.0,3336.0,3609.0,0.0,0.0,0.0,0.0,21061.0
2,Alexandria,0.0,0.0,3638.0,0.0,28682.0,6786.0,3388.0,0.0,2939.0,...,0.0,0.0,17654.0,10323.0,4527.0,0.0,0.0,0.0,0.0,39332.0
3,Alleghany,0.0,0.0,75.0,0.0,2039.0,82.0,140.0,0.0,51.0,...,0.0,0.0,3190.0,102.0,189.0,0.0,0.0,0.0,0.0,2837.0
4,Amelia,0.0,0.0,46.0,0.0,932.0,74.0,746.0,0.0,34.0,...,0.0,0.0,2369.0,101.0,922.0,0.0,0.0,0.0,0.0,1358.0


In [43]:
Virginia_County_Vaccine_Race.columns

Index(['County', 'Black_Not Reported', 'Latino_Not Reported',
       'Other Race_At Least One Dose', 'Two or more races_Fully Vaccinated',
       'Not Reported_Fully Vaccinated', 'Latino_Fully Vaccinated',
       'Black_Fully Vaccinated', 'Other Race_Not Reported',
       'Other Race_Fully Vaccinated',
       'Asian or Pacific Islander_Fully Vaccinated',
       'Native American_At Least One Dose', 'Native American_Fully Vaccinated',
       'White_Fully Vaccinated', 'Asian or Pacific Islander_At Least One Dose',
       'Native American_Not Reported', 'Two or more races_At Least One Dose',
       'White_At Least One Dose', 'Latino_At Least One Dose',
       'Black_At Least One Dose', 'Two or more races_Not Reported',
       'Asian or Pacific Islander_Not Reported', 'White_Not Reported',
       'Not Reported_Not Reported', 'Not Reported_At Least One Dose'],
      dtype='object')

In [44]:
Virginia_County_Vaccine_Race.loc[0]

County                                         Accomack
Black_Not Reported                                  0.0
Latino_Not Reported                                 0.0
Other Race_At Least One Dose                       35.0
Two or more races_Fully Vaccinated                  0.0
Not Reported_Fully Vaccinated                    2654.0
Latino_Fully Vaccinated                           492.0
Black_Fully Vaccinated                           2018.0
Other Race_Not Reported                             0.0
Other Race_Fully Vaccinated                        26.0
Asian or Pacific Islander_Fully Vaccinated         79.0
Native American_At Least One Dose                  25.0
Native American_Fully Vaccinated                   25.0
White_Fully Vaccinated                           6422.0
Asian or Pacific Islander_At Least One Dose        95.0
Native American_Not Reported                        0.0
Two or more races_At Least One Dose                 0.0
White_At Least One Dose                         

In [36]:
Virginia_County_Vaccine_Race['County'].values

array(['Accomack', 'Albemarle', 'Alexandria', 'Alleghany', 'Amelia',
       'Amherst', 'Appomattox', 'Arlington', 'Augusta', 'Bath', 'Bedford',
       'Bland', 'Botetourt', 'Bristol', 'Brunswick', 'Buchanan',
       'Buckingham', 'Buena Vista City', 'Campbell', 'Caroline',
       'Carroll', 'Charles City', 'Charlotte', 'Charlottesville',
       'Chesapeake', 'Chesterfield', 'Clarke', 'Colonial Heights',
       'Covington', 'Craig', 'Culpeper', 'Cumberland', 'Danville',
       'Dickenson', 'Dinwiddie', 'Emporia', 'Essex', 'Fairfax',
       'Fairfax City', 'Falls Church', 'Fauquier', 'Floyd', 'Fluvanna',
       'Franklin City', 'Franklin County', 'Frederick', 'Fredericksburg',
       'Galax', 'Giles', 'Gloucester', 'Goochland', 'Grayson', 'Greene',
       'Greensville', 'Halifax', 'Hampton', 'Hanover', 'Harrisonburg',
       'Henrico', 'Henry', 'Highland', 'Hopewell', 'Isle of Wight',
       'James City', 'King George', 'King William', 'King and Queen',
       'Lancaster', 'Lee', 'Lexing

In [39]:
# print(len(L))

import numpy as np


virginia_name_dict = {
 'Colonial Heights, Virginia': ['Colonial Heights city, Virginia'],
 
 'Portsmouth, Virginia': ['Portsmouth city, Virginia'],
    
 'Roanoke City, Virginia': ['Roanoke city, Virginia'],
    
 'Charlottesville, Virginia': ['Charlottesville city, Virginia'],
    
 'Waynesboro, Virginia': ['Waynesboro city, Virginia'],
    
 'Virginia Beach, Virginia': ['Virginia Beach city, Virginia'],
    
 'Suffolk, Virginia': ['Suffolk city, Virginia'],
    
 'Williamsburg, Virginia': ['Williamsburg city, Virginia'],
    
 'Roanoke County, Virginia': ['Roanoke, Virginia'],
    
 'Falls Church, Virginia': ['Falls Church city, Virginia'],
    
 'Franklin County, Virginia': ['Franklin, Virginia'],
    
 'Buena Vista City, Virginia': ['Buena Vista city, Virginia'],
 
 'Lynchburg, Virginia': ['Lynchburg city, Virginia'],
    
 'Hampton, Virginia': ['Hampton city, Virginia'],
    
 'Fredericksburg, Virginia': ['Fredericksburg city, Virginia'],
    
 'Staunton, Virginia': ['Staunton city, Virginia'],
    
 'Galax, Virginia': ['Galax city, Virginia'],
    
 'Richmond County, Virginia': ['Richmond, Virginia'],
    
 'Petersburg, Virginia': ['Petersburg city, Virginia'],
    
 'Harrisonburg, Virginia': ['Harrisonburg city, Virginia'],
    
 'Richmond City, Virginia': ['Richmond city, Virginia'],
    
 'Covington, Virginia': ['Covington city, Virginia'],
    
 'Norfolk, Virginia': ['Norfolk city, Virginia'],
    
 'Fairfax City, Virginia': ['Fairfax city, Virginia'],
    
 'Bristol, Virginia': ['Bristol city, Virginia'],
    
 'Isle Of Wight, Virginia': ['Isle of Wight, Virginia'],
    
 'Lexington, Virginia': ['Lexington city, Virginia'],
    
 'Hopewell, Virginia': ['Hopewell city, Virginia'],
    
 'Alexandria, Virginia': ['Alexandria city, Virginia'],
    
 'Manassas City, Virginia': ['Manassas city, Virginia'],
    
 'Winchester, Virginia': ['Winchester city, Virginia'],
    
 'Manassas Park, Virginia': ['Manassas Park city, Virginia'],
    
 'Poquoson, Virginia': ['Poquoson city, Virginia'],
    
 'Emporia, Virginia': ['Emporia city, Virginia'],
    
 'Radford, Virginia': ['Radford city, Virginia'],
    
 'Salem, Virginia': ['Salem city, Virginia'],
    
 'Chesapeake, Virginia': ['Chesapeake city, Virginia'],
    
 'King And Queen, Virginia': ['King and Queen, Virginia'],
    
 'Franklin City, Virginia': ['Franklin city, Virginia'],
    
 'Norton, Virginia': ['Norton city, Virginia'],
  
 'Danville, Virginia': ['Danville city, Virginia'],
    
 'Newport News, Virginia': ['Newport News city, Virginia'],
    
 'Martinsville, Virginia': ['Martinsville city, Virginia']}

RenameCounties = {k.replace(', Virginia', ''): v[0].replace(', Virginia', '') for k, v in virginia_name_dict.items()}

state = 'Virginia' ## 

new_L = []


for idx, row in Virginia_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    
    new_d['Numb_White'] = d['White_At Least One Dose']
    
    new_d['Numb_Black'] = d['Black_At Least One Dose']
    
    new_d['Numb_Asian'] = d['Asian or Pacific Islander_At Least One Dose']
    
    
    new_d['FullNumb_White'] = d['White_Fully Vaccinated']
    
    new_d['FullNumb_Black'] = d['Black_Fully Vaccinated']
    
    new_d['FullNumb_Asian'] = d['Asian or Pacific Islander_Fully Vaccinated']
    
    
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate
    
    
    Rate = T['FullNumb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['FullRate_' + Race] = Rate

T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

    

Not included Counties ['Out-of-state, Virginia', 'Not Reported, Virginia']
(133, 25)


,State,County,Geographic Area Name,Date,FullNumb_Asian,FullNumb_Black,FullNumb_White,FullRate_Asian,FullRate_Black,FullRate_White,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Virginia,Accomack,"Accomack, Virginia",NaN,79.0,2018.0,6422.0,0.307393,0.216896,0.293255,...,0.321796,239,447,257,348,9304,9632,21899,22411,32316
1,Virginia,Albemarle,"Albemarle, Virginia",NaN,1164.0,2896.0,31052.0,0.192365,0.273208,0.347384,...,0.415872,383,1000,6051,7189,10600,12008,89388,92002,109330
2,Virginia,Alexandria city,"Alexandria city, Virginia",NaN,1389.0,3388.0,14807.0,0.133841,0.093359,0.139256,...,0.166032,998,2243,10378,12886,36290,38626,106329,110857,159428
3,Virginia,Alleghany,"Alleghany, Virginia",NaN,14.0,140.0,2598.0,0.304348,0.200573,0.188493,...,0.231445,44,119,46,74,698,890,13783,14040,14860
4,Virginia,Amelia,"Amelia, Virginia",NaN,20.0,746.0,1893.0,0.250000,0.277530,0.188358,...,0.235721,79,161,80,124,2688,2827,10050,10264,13145


In [40]:
# Virginia	Accomack	Accomack, Virginia	NaN	63.0	1704.0	5764.0	0.245136	0.183147	0.263208	239	447	257
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana', 'Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,133.0,0.186399,0.072396,0.051606,0.132377,0.170360,0.229512,0.437086
1,FullRate_White,133.0,0.210906,0.063200,0.089883,0.163396,0.200643,0.243229,0.428161
2,Rate_Black,133.0,0.226668,0.087424,0.067661,0.163379,0.211537,0.272340,0.608696
3,Rate_White,133.0,0.244373,0.069999,0.105500,0.185471,0.235721,0.283663,0.479973
4,Rate_Parity,133.0,0.017705,0.082831,-0.423878,-0.012011,0.028370,0.060885,0.275374
5,FullRate_Parity,133.0,0.024507,0.068099,-0.223975,-0.007011,0.030524,0.060157,0.292251


# All in One

## Read Previous Vaccine Data

In [38]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: CountyVaccine/RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [39]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [40]:
# ######################################

# # Today_Date = str(datetime.now().date())
# print(Today_Date)
# ######################################


# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']
    
    
    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [41]:

# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']


# DF[DF['Rate_Black'] > 0.8][data_cols]

## Save Data

In [41]:

import pickle 

path = 'CountyVaccine/Workplace/STATE_VACCINE_' + Today_Date + '_Autonomy.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
CountyVaccine/Workplace/STATE_VACCINE_2021-05-20_Autonomy.p
If yes, please enter: Yes
Yes
Successfully Saved!
